In [92]:
def faulty_solve(filename):
    map = open(filename).read().splitlines()
    ends = {
        c: (i, j)
        for i, line in enumerate(map)
        for j, c in enumerate(line)
        if c == "S" or c == "E"
    }
    isPath = lambda p: map[p[0]][p[1]] == "." or map[p[0]][p[1]] == "E"
    rows = len(map)
    cols = len(map[0])
    max_len = int(1.15*(rows + cols))
    print(rows, cols, max_len)
    def step_turn(R):
        if ends["E"] == R["p"]:
            return [R]
        if R["s"] > max_len:
            return []
        clones = []
        dirs = [R["d"], (-1, 0), (1, 0)] if R["d"][0] == 0 else [R["d"], (0, 1), (0, -1)]
        for d in dirs:
            nxt_p = tuple(sum(x) for x in zip(R["p"], d))
            if isPath(nxt_p) and not nxt_p in R["pth"]:
                clones.extend(
                    step_turn(
                        {
                            "s": R["s"] + 1,
                            "t": R["t"] + (R["d"] != d),
                            "d": d,
                            "p": nxt_p,
                            "pth": {tuple(nxt_p), *R["pth"]},
                        }
                    )
                )             
        return clones

    clones = step_turn(
        {"s": 0, "t": 0, "d": (0, 1), "p": ends["S"], "pth": {tuple(ends["S"])}}
    )
    return min([clone["s"] +clone["t"]*1000 for clone in clones])
print(f"part 1 - test: {faulty_solve("./test16.txt")}")
print(f"part 1 - test-2: {faulty_solve("./test16-2.txt")}")


15 15 34
part 1 - test: 10028
17 17 39
part 1 - test-2: 14040


In [90]:


def solved(filename):
    map = open(filename).read().splitlines()
    ends = {
        c: (i, j)
        for i, line in enumerate(map)
        for j, c in enumerate(line)
        if c == "S" or c == "E"
    }
    nodes = tuple((i,j) for i, line in enumerate(map) for j, c in enumerate(line) if c != "#")
    get_dirs = lambda d: [d, (1,0), (-1,0)] if d[0] == 0 else [d, (0,1), (0,-1)]
    get_nbrs =lambda i,j,d: tuple(n for n in [((i+di,j+dj),(di,dj)) for di, dj in get_dirs(d)] if map[n[0][0]][n[0][1]] != "#")
    graph = {((i,j), d): get_nbrs(i,j, d) for i,j in nodes for d in [(1,0),(-1,0),(0,1),(0,-1)]}
    visited = {g: False for g in graph}
    dists = {g: float("inf") for g in graph}
    dists[(ends["S"],(0,1))] = 0
    pq = [(0, (ends["S"], (0,1)))]

    while pq:
        dist, node =  pq.pop()
        if visited[node]:
            continue
        visited[node] = True
        for nbr in graph[node]:
            nbr_dist  = dist + 1*(nbr[0] != node[0]) + 1000*(nbr[1] != node[1])
            if nbr_dist < dists[nbr]:
                dists[nbr] = nbr_dist
                pq.append((nbr_dist, nbr))
        pq.sort(key=lambda v: v[0], reverse=True)

    return min([dists[(ends["E"], d)] for d in [(-1,0),(0,1)]])


        
        
            
    


print(f"part 1 - test: {solved("./test16.txt")}")
print(f"part 1 - test-2: {solved("./test16-2.txt")}")
print(f"part 1 - result: {solved("./day16.txt")}")

part 1 - test: 7036
part 1 - test-2: 11048
part 1 - result: 91464


In [ ]:


def solved2(filename):
    map = open(filename).read().splitlines()
    ends = {
        c: (i, j)
        for i, line in enumerate(map)
        for j, c in enumerate(line)
        if c == "S" or c == "E"
    }
    nodes = tuple((i,j) for i, line in enumerate(map) for j, c in enumerate(line) if c != "#")
    get_dirs = lambda d: [d, (1,0), (-1,0)] if d[0] == 0 else [d, (0,1), (0,-1)]
    get_nbrs =lambda i,j,d: tuple(n for n in [((i+di,j+dj),(di,dj)) for di, dj in get_dirs(d)] if map[n[0][0]][n[0][1]] != "#")
    graph = {((i,j), d): get_nbrs(i,j, d) for i,j in nodes for d in [(1,0),(-1,0),(0,1),(0,-1)]}
    visited = {g: False for g in graph}
    dists = {g: float("inf") for g in graph}
    dists[(ends["S"],(0,1))] = 0
    pq = [(0, (ends["S"], (0,1)))]

    while pq:
        dist, node =  pq.pop()
        if visited[node]:
            continue
        visited[node] = True
        for nbr in graph[node]:
            nbr_dist  = dist + 1*(nbr[0] != node[0]) + 1000*(nbr[1] != node[1])
            if nbr_dist < dists[nbr]:
                dists[nbr] = nbr_dist
                pq.append((nbr_dist, nbr))
        pq.sort(key=lambda v: v[0], reverse=True)

    min_path =  min([dists[(ends["E"], d)] for d in [(-1,0),(0,1)]])
    target_node = [(ends["E"], d) for d in [(-1,0),(0,1)] if dists[(ends["E"], d)] ==  min_path][0]
    predecessors = {node: set() for node in graph}
    for node, dist in dists.items():
            for nbr in graph[node]:
                if dists[nbr] == dist + 1*(nbr[0] != node[0]) + 1000*(nbr[1] != node[1]):
                     predecessors[nbr].add(node)

    def get_seats(target_node):
        seats = set()
        seats.add(target_node[0])
        for node in predecessors[target_node]:
            seats.update(get_seats(node))
        return seats
    seats = get_seats(target_node)
              
    sol_map  = [list(line) for line in map.copy()]
    # for p in seats:
    #     sol_map[p[0]][p[1]] = "O"
    # for line in sol_map: print("".join(line))
    return len(seats)


print(f"part 1 - test: {solved2("./test16.txt")}")
print(f"part 1 - test-2: {solved2("./test16-2.txt")}")
print(f"part 1 - result: {solved2("./day16.txt")}")

part 1 - test: 45
part 1 - test-2: 64
part 1 - result: 494
